In [1]:
import numpy as np
import pyvista as pv
import torch
import sys
import matplotlib.pyplot as plt
from scipy import interpolate

In [2]:
import Multi_Species_Cu_Si_noAr_etching

In [3]:
DXsec_Al = np.load('./DXsec_Al_1e8.npy')

In [21]:
film = np.zeros((100, 100, 1000, 3))

bottom = 900
height = 940

density = 10

sphere = np.ones((100, 100, 1000), dtype=bool)

radius = 30

center = 50
for i in range(sphere.shape[0]):
    for j in range(sphere.shape[1]):
        if np.abs(i-center)*np.abs(i-center) + np.abs(j-center)*np.abs(j-center) < radius*radius:
            sphere[i, j, bottom:height] = 0

film[sphere, 2] = density
film[:, :, height:, :] = 0
film[:, :, 0:bottom, 0] = -density # bottom
film[:, :, 0:bottom, 1] = 0 # bottom
film[:, :, 0:bottom, 2] = 0 # bottom
# film[:, :30, bottom:height, 1] = 10
# film[:, -30:, bottom:height, 1] = 10


In [ ]:
depo1 = torch.Tensor(film[:50, :,:, 2]==density).to_sparse()
depo1 = depo1.indices().numpy().T

substrute = torch.Tensor(film[:50, :,:, 0]==-density).to_sparse()
substrute = substrute.indices().numpy().T
depomesh = pv.PolyData(depo1)
depomesh["radius"] = np.ones(depo1.shape[0])*0.5
geom = pv.Box()

submesh = pv.PolyData(substrute)
submesh["radius"] = np.ones(substrute.shape[0])*0.5

# Progress bar is a new feature on master branch
depoglyphed = depomesh.glyph(scale="radius", geom=geom) # progress_bar=True)
subglyphed = submesh.glyph(scale="radius", geom=geom) # progress_bar=True)

p = pv.Plotter()
# p.add_mesh(depoglyphed, color='cyan')
p.add_mesh(subglyphed, color='dimgray')
p.enable_eye_dome_lighting()
p.show()

In [4]:
film = np.zeros((100, 100, 160, 3))

bottom = 100
height = 140

density = 10

sphere = np.ones((100, 100, 160), dtype=bool)

radius = 30

center = 50
for i in range(sphere.shape[0]):
    for j in range(sphere.shape[1]):
        if np.abs(i-center)*np.abs(i-center) + np.abs(j-center)*np.abs(j-center) < radius*radius:
            sphere[i, j, bottom:height] = 0

film[sphere, 2] = density
film[:, :, height:, :] = 0
film[:, :, 0:bottom, 0] = -density # bottom
film[:, :, 0:bottom, 1] = 0 # bottom
film[:, :, 0:bottom, 2] = 0 # bottom

In [ ]:
etchfilm[10, 10, 30]

In [5]:
etchfilm = film

In [6]:
N = int(1e7)

In [7]:
logname = 'Multi_species_benchmark_0729'
testEtch = Multi_Species_Cu_Si_noAr_etching.etching(mirror=True,inputMethod='bunch', pressure_pa=0.001, temperature=300, chamberSize=etchfilm.shape,
                    depoThick=bottom - 40, center_with_direction=np.array([[35,100,75]]), 
                    range3D=np.array([[0, 70, 0, 100, 0, 150]]), InOrOut=[1], yield_hist=np.array([None]),
                    reaction_type=False, param = [1.6, -0.7], N = N, 
                    sub_xy=[0,0], film=etchfilm, n=1, cellSize=etchfilm.shape, 
                    celllength=1e-5, kdtreeN=5, tstep=1e-5,
                    substrateTop=30,posGeneratorType='top', logname=logname)

In [ ]:
testEtch.parcel[:10]

In [8]:
etching = testEtch.depo_position_increase_cosVel_normal(125, N, 2e-2, 10)

running: 100%|██████████████████████████████████████████████████████| 100/100 [26:10<00:00, 15.7s/B]


In [10]:
depo1 = torch.Tensor(etchfilm[:50, :, :, 1]!=0).to_sparse()
depo1 = depo1.indices().numpy().T

substrute = torch.Tensor(etchfilm[:50, :, :,0]!=0).to_sparse()
substrute = substrute.indices().numpy().T

mask = torch.Tensor(etchfilm[:50, :, :,2]!=0).to_sparse()
mask = mask.indices().numpy().T

maskmesh = pv.PolyData(mask)
maskmesh["radius"] = np.ones(mask.shape[0])*0.5

depomesh = pv.PolyData(depo1)
depomesh["radius"] = np.ones(depo1.shape[0])*0.5
geom = pv.Box()

submesh = pv.PolyData(substrute)
submesh["radius"] = np.ones(substrute.shape[0])*0.5

# Progress bar is a new feature on master branch
maskglyphed = maskmesh.glyph(scale="radius", geom=geom) # progress_bar=True)
depoglyphed = depomesh.glyph(scale="radius", geom=geom) # progress_bar=True)
subglyphed = submesh.glyph(scale="radius", geom=geom) # progress_bar=True)

p = pv.Plotter()
p.add_mesh(maskglyphed, color='green')
# p.add_mesh(depoglyphed, color='cyan')
p.add_mesh(subglyphed, color='dimgray')
p.enable_eye_dome_lighting()
p.show()

c:\Users\urcs049\AppData\Local\anaconda3\Lib\site-packages\pyvista\core\utilities\points.py:55: UserWarning: Points is not a float type. This can cause issues when transforming or applying filters. Casting to ``np.float32``. Disable this by passing ``force_float=False``.
  warnings.warn(
c:\Users\urcs049\AppData\Local\anaconda3\Lib\site-packages\pyvista\core\filters\data_set.py:2386: UserWarning: No vector-like data to use for orient. orient will be set to False.
  warnings.warn("No vector-like data to use for orient. orient will be set to False.")


Widget(value='<iframe src="http://localhost:52600/index.html?ui=P_0x2485d068e00_1&reconnect=auto" class="pyvis…

In [ ]:
x_erosion_np = testEtch.parcel[:,:3][1:,1]
y_erosion_np = testEtch.parcel[:,:3][1:,2]

xbins = np.linspace(-testEtch.parcel[:,:3][:, 1].max()/2,testEtch.parcel[:,:3][:, 1].max()*1.5,200)
ybins = np.linspace(-testEtch.parcel[:,:3][:, 2].max()/2,testEtch.parcel[:,:3][:, 2].max()*1.5,200)
# ybins = np.linspace(1.815, 2.415, 200)
x_bins = np.histogram(x_erosion_np, bins=xbins)
y_bins = np.histogram(y_erosion_np, bins=ybins)

H, xedges, yedges = np.histogram2d(x_erosion_np, y_erosion_np, bins=(xbins, ybins))


fig, ax = plt.subplots(figsize=(8,8))
im = ax.imshow(H.T, interpolation='nearest', origin='lower',
        extent=[xedges[0], xedges[-1], yedges[0], yedges[-1]])
ax.set_title('collision position in trench(side view)')
ax.set_xlabel('x')
ax.set_ylabel('y')
fig.colorbar(im, ax=ax, label='Events')
plt.show()

In [ ]:
x_erosion_np = testEtch.parcel[:,6:9][1:,1]
y_erosion_np = testEtch.parcel[:,6:9][1:,2]

xbins = np.linspace(-testEtch.parcel[:,6:9][:, 1].max()/2,testEtch.parcel[:,6:9][:, 1].max()*1.5,200)
ybins = np.linspace(-testEtch.parcel[:,6:9][:, 2].max()/2,testEtch.parcel[:,6:9][:, 2].max()*1.5,200)
# ybins = np.linspace(1.815, 2.415, 200)
x_bins = np.histogram(x_erosion_np, bins=xbins)
y_bins = np.histogram(y_erosion_np, bins=ybins)

H, xedges, yedges = np.histogram2d(x_erosion_np, y_erosion_np, bins=(xbins, ybins))


fig, ax = plt.subplots(figsize=(8,8))
im = ax.imshow(H.T, interpolation='nearest', origin='lower',
        extent=[xedges[0], xedges[-1], yedges[0], yedges[-1]])
ax.set_title('collision position in trench(side view)')
ax.set_xlabel('x')
ax.set_ylabel('y')
fig.colorbar(im, ax=ax, label='Events')
plt.show()

In [ ]:
etchfilm[50, 50:55, bottom-3:bottom+2, :]

In [4]:
import Multi_Species_bosch_etching_noMirrorSymmtry

In [5]:
logname = 'Multi_species_benchmark_0920'
testEtch = Multi_Species_bosch_etching_noMirrorSymmtry.etching(inputMethod='bunch', depo_or_etching='etching', 
                etchingPoint = np.array([center, center, 37]),depoPoint = np.array([center, center, 37]),
                density=density, center_with_direction=np.array([[35,100,75]]), 
                range3D=np.array([[0, 70, 0, 100, 0, 150]]), InOrOut=[1], yield_hist=np.array([None]),
                reaction_type=False, param = [1.6, -0.7],
                sub_xy=[0,0], film=etchfilm, n=1, cellSize=etchfilm.shape, 
                celllength=1e-5, kdtreeN=5, tstep=1e-5,
                substrateTop=bottom,posGeneratorType='top', logname=logname)

In [14]:
# cicle = 100

# for i in range(cicle):
#     step1 = testEtch.inputParticle('maxwell', 0, int(2e4),int(4e7), 10)
#     depth = bottom - step1[1] 
#     np.save('./bosch_data_0921/bosch_sf_step_{}_sf'.format(i), etchfilm)
#     step2 = testEtch.inputParticle('maxwell', 1, int(2e4),int(1e7), 10)
#     indice_c4 = etchfilm[:, :, :, 1] >= 10
#     etchfilm[indice_c4, 1] = 10
#     np.save('./bosch_data_0921/bosch_sf_step_{}_c4f8'.format(i), etchfilm)
#     step3 = testEtch.inputParticle('updown', 2, int(2e4),int(4e6)+int(depth/(4e6/2e4)*4e6), 10)
#     np.save('./bosch_data_0921/bosch_sf_step_{}_Ar'.format(i), etchfilm)

In [ ]:
step3 = testEtch.inputParticle('updown', 2, int(2e4),int(3e6), 10)

In [ ]:
depo1 = torch.Tensor(etchfilm[:50, :, bottom-100:bottom+10, 1]!=0).to_sparse()
depo1 = depo1.indices().numpy().T

substrute = torch.Tensor(etchfilm[:50, :, bottom-100:bottom+10,0]!=0).to_sparse()
substrute = substrute.indices().numpy().T

mask = torch.Tensor(etchfilm[:50, :, bottom-100:bottom+10,2]!=0).to_sparse()
mask = mask.indices().numpy().T

maskmesh = pv.PolyData(mask)
maskmesh["radius"] = np.ones(mask.shape[0])*0.5

depomesh = pv.PolyData(depo1)
depomesh["radius"] = np.ones(depo1.shape[0])*0.5
geom = pv.Box()

submesh = pv.PolyData(substrute)
submesh["radius"] = np.ones(substrute.shape[0])*0.5

# Progress bar is a new feature on master branch
maskglyphed = maskmesh.glyph(scale="radius", geom=geom) # progress_bar=True)
depoglyphed = depomesh.glyph(scale="radius", geom=geom) # progress_bar=True)
subglyphed = submesh.glyph(scale="radius", geom=geom) # progress_bar=True)

p = pv.Plotter()
p.add_mesh(maskglyphed, color='green')
# p.add_mesh(depoglyphed, color='cyan')
p.add_mesh(subglyphed, color='dimgray')
p.enable_eye_dome_lighting()
p.show()

In [ ]:
cicle = 100

for i in range(cicle):
    step1 = testEtch.inputParticle('maxwell', 0, int(7e4),int(5e7), 10)
    depth = bottom - step1[1] 
    np.save('./bosch_data_0926_ratio08/bosch_sf_step_{}_sf'.format(i), etchfilm)
    step2 = testEtch.inputParticle('maxwell', 1, int(3e4),int(2e7), 10)
    indice_c4 = etchfilm[:, :, :, 1] >= 10
    etchfilm[indice_c4, 1] = 10
    np.save('./bosch_data_0926_ratio08/bosch_sf_step_{}_c4f8'.format(i), etchfilm)
    step3 = testEtch.inputParticle('updown', 2, int(2e4),int(4e6)+int(depth/(3e6/2e4)*3e6), 10)
    np.save('./bosch_data_0926_ratio08/bosch_sf_step_{}_Ar'.format(i), etchfilm)

In [13]:
np.save('bosch_sf_step11', etchfilm)

In [24]:
etching = np.load('./bosch_data/bosch_sf_step_0_sf.npy')

In [29]:
etching = np.load('./bosch_data/bosch_sf_step_0_c4f8.npy')

In [27]:
etching = np.load('./bosch_data/bosch_sf_step_0_Ar.npy')

In [ ]:
depo1 = torch.Tensor(etching[:, :, bottom-50:, 1]!=0).to_sparse()
depo1 = depo1.indices().numpy().T

substrute = torch.Tensor(etching[:, :, bottom-50:,0]!=0).to_sparse()
substrute = substrute.indices().numpy().T
depomesh = pv.PolyData(depo1)
depomesh["radius"] = np.ones(depo1.shape[0])*0.5
geom = pv.Box()

submesh = pv.PolyData(substrute)
submesh["radius"] = np.ones(substrute.shape[0])*0.5

# Progress bar is a new feature on master branch
depoglyphed = depomesh.glyph(scale="radius", geom=geom) # progress_bar=True)
subglyphed = submesh.glyph(scale="radius", geom=geom) # progress_bar=True)

p = pv.Plotter()
p.add_mesh(depoglyphed, color='cyan')
p.add_mesh(subglyphed, color='dimgray')
p.enable_eye_dome_lighting()
p.show()

In [ ]:
depo1 = torch.Tensor(etching[:, :50, bottom-200:bottom+50, 1]!=0).to_sparse()
depo1 = depo1.indices().numpy().T

substrute = torch.Tensor(etching[:, :50, bottom-200:bottom+50,0]!=0).to_sparse()
substrute = substrute.indices().numpy().T
depomesh = pv.PolyData(depo1)
depomesh["radius"] = np.ones(depo1.shape[0])*0.5
geom = pv.Box()

submesh = pv.PolyData(substrute)
submesh["radius"] = np.ones(substrute.shape[0])*0.5

# Progress bar is a new feature on master branch
depoglyphed = depomesh.glyph(scale="radius", geom=geom) # progress_bar=True)
subglyphed = submesh.glyph(scale="radius", geom=geom) # progress_bar=True)

p = pv.Plotter()
p.add_mesh(depoglyphed, color='cyan')
p.add_mesh(subglyphed, color='dimgray')
p.enable_eye_dome_lighting()
p.show()

In [ ]:
depo1 = torch.Tensor(etching[:, :, bottom-400:bottom-200, 1]!=0).to_sparse()
depo1 = depo1.indices().numpy().T

substrute = torch.Tensor(etching[:, :, bottom-400:bottom-200,0]!=0).to_sparse()
substrute = substrute.indices().numpy().T
depomesh = pv.PolyData(depo1)
depomesh["radius"] = np.ones(depo1.shape[0])*0.5
geom = pv.Box()

submesh = pv.PolyData(substrute)
submesh["radius"] = np.ones(substrute.shape[0])*0.5

# Progress bar is a new feature on master branch
depoglyphed = depomesh.glyph(scale="radius", geom=geom) # progress_bar=True)
subglyphed = submesh.glyph(scale="radius", geom=geom) # progress_bar=True)

p = pv.Plotter()
p.add_mesh(depoglyphed, color='cyan')
p.add_mesh(subglyphed, color='dimgray')
p.enable_eye_dome_lighting()
p.show()

In [ ]:
depo1 = torch.Tensor(etching[:, :75, bottom-400:bottom-200, 1]!=0).to_sparse()
depo1 = depo1.indices().numpy().T

substrute = torch.Tensor(etching[:, :75, bottom-400:bottom-200,0]!=0).to_sparse()
substrute = substrute.indices().numpy().T
depomesh = pv.PolyData(depo1)
depomesh["radius"] = np.ones(depo1.shape[0])*0.5
geom = pv.Box()

submesh = pv.PolyData(substrute)
submesh["radius"] = np.ones(substrute.shape[0])*0.5

# Progress bar is a new feature on master branch
depoglyphed = depomesh.glyph(scale="radius", geom=geom) # progress_bar=True)
subglyphed = submesh.glyph(scale="radius", geom=geom) # progress_bar=True)

p = pv.Plotter()
p.add_mesh(depoglyphed, color='cyan')
p.add_mesh(subglyphed, color='dimgray')
p.enable_eye_dome_lighting()
p.show()